In [1]:
!pip install tensorflow_hub


In [2]:
import tensorflow as tf
import tensorflow_hub as hub

# MoveNet 모델 로드 (저장 없이 직접 사용)
model = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
movenet = model.signatures['serving_default']

def get_keypoints(frame):
    img = cv2.resize(frame, (192, 192))  # Lightning 모델은 192x192 입력
    img = img.astype(np.float32)
    img = np.expand_dims(img, axis=0)
    input_image = tf.convert_to_tensor(img)
    outputs = movenet(input_image)
    keypoints = outputs['output_0'].numpy()[0, 0]  # (17, 3)
    return keypoints


In [3]:
!pip install opencv-python

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import requests
import threading
import torch
import torch.nn as nn
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn
from fastapi.middleware.cors import CORSMiddleware

# --- 1. 낙상 분류 모델 정의 및 로드 ---
class FallClassifierWithAttention(nn.Module):
    def __init__(self, input_dim=51, hidden_dim=256, num_layers=3, output_dim=3, dropout=0.5):
        super(FallClassifierWithAttention, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers,
                            batch_first=True, bidirectional=True, dropout=dropout)
        self.attention = nn.Linear(hidden_dim * 2, 1)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        attn_scores = self.attention(lstm_out)
        attn_weights = torch.softmax(attn_scores, dim=1)
        context = torch.sum(attn_weights * lstm_out, dim=1)
        out = self.dropout(context)
        out = self.fc(out)
        return out

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FallClassifierWithAttention().to(device)
model.load_state_dict(torch.load("../model/best_model.pt", map_location=device))
model.eval()

# --- 2. FastAPI 서버 설정 ---
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

class KeypointSequence(BaseModel):
    sequence: list  # (100, 51) 2차원 리스트

@app.post("/predict")
async def predict_pose(data: KeypointSequence):
    x = np.array(data.sequence, dtype=np.float32)
    x = torch.tensor(x).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(x)
        probs = torch.softmax(output, dim=1).cpu().numpy()[0]
    return {"probs": probs.tolist()}

def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8001)

# --- 3. MoveNet 모델 로드 ---
model_movenet = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
movenet = model_movenet.signatures['serving_default']

def get_keypoints(frame):
    img = cv2.resize(frame, (192, 192))
    img = img.astype(np.int32)
    img = np.expand_dims(img, axis=0)
    input_image = tf.convert_to_tensor(img, dtype=tf.int32)
    outputs = movenet(input_image)
    keypoints = outputs['output_0'].numpy()[0, 0]
    keypoints_xyc = np.stack([
        keypoints[:, 1],  # x
        keypoints[:, 0],  # y
        keypoints[:, 2]   # confidence
    ], axis=1).flatten()
    return keypoints_xyc

# --- 4. 메인 실행 ---
if __name__ == "__main__":
    # 기존 서버가 실행 중이면 충돌 날 수 있음 → 터미널 재시작 권장
    server_thread = threading.Thread(target=run_server, daemon=True)
    server_thread.start()
    print("FastAPI 서버가 백그라운드에서 시작되었습니다 (http://localhost:8001)")

    cap = cv2.VideoCapture(0)
    sequence = []

    while True:
        ret, frame = cap.read()
        if not ret:
            print("웹캠 프레임을 읽을 수 없습니다.")
            break

        try:
            keypoints = get_keypoints(frame)
            sequence.append(keypoints.tolist())
            from sklearn.preprocessing import StandardScaler
            def normalize_sequence(seq_2d):
                scaler = StandardScaler()
                return scaler.fit_transform(seq_2d)
            if len(sequence) == 100:
                try:
                    seq_array = np.array(sequence)
                    norm_seq = normalize_sequence(seq_array)
                    data = {"sequence": sequence}
                    
                    res = requests.post("http://localhost:8001/predict", json=data, timeout=3)
                    probs = res.json()["probs"]

                    if probs[2] > 0.7:
                        status = "Fall"
                    elif probs[1] > 0.5:
                        status = "Suspicious"
                    elif probs[0] > 0.7:
                        status = "Normal"
                    else:
                        status = "Uncertain"

                    print(f"예측 결과: {status} (확률: {probs})")
                    requests.post("http://localhost:8080/fall", json={"status": status})
                except Exception as e:
                    print(f"추론 서버 연결 실패: {e}")
                sequence = []
        except Exception as e:
            print(f"관절 추출 에러: {e}")

        cv2.imshow('Webcam', frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()


FastAPI 서버가 백그라운드에서 시작되었습니다 (http://localhost:8001)


INFO:     Started server process [34956]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 8001): [winerror 10048] 각 소켓 주소(프로토콜/네트워크 주소/포트)는 하나만 사용할 수 있습니다
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


INFO:     127.0.0.1:1517 - "POST /predict HTTP/1.1" 200 OK
예측 결과: Suspicious (확률: [1.5626914319000207e-05, 0.9955345392227173, 0.004449807573109865])
INFO:     127.0.0.1:1526 - "POST /predict HTTP/1.1" 200 OK
예측 결과: Suspicious (확률: [1.0971447409247048e-05, 0.9960617423057556, 0.00392732210457325])
INFO:     127.0.0.1:1546 - "POST /predict HTTP/1.1" 200 OK
예측 결과: Suspicious (확률: [0.03416558355093002, 0.9657483696937561, 8.606762276031077e-05])


KeyboardInterrupt: 

: 